# How to fool DeepMind Perceiver in 10 minutes with adversarial examples


👉🏽 [DeepMind Perceiver](https://www.deepmind.com/blog/building-architectures-that-can-handle-the-worlds-data) was released in 2021. It's a transformer-based model that can handle data from arbitrary settings, including images, videos, text, or audio. This model is one solid step toward truly general architectures, as opposed to task-specific, that can process arbitrary sequences of inputs and outputs.

👉🏽 In this notebook, we will generate and adversarial example that manages to fool this model.

👉🏽 We will use the implementation of the Perceiver available in [HuggingFace](https://huggingface.co/deepmind/vision-perceiver-learned#how-to-use) model repository.

<a href="https://colab.research.google.com/github/Paulescu/fooling_deepmind/blob/main/how_to_fool_deepmind_perceiver_in_10_minutes.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" width="200"></a>

In [ ]:
!pip install transformers cleverhans torch

## Load the `PerceiverIO` model

![](images/pipeline.png)

In [ ]:
from transformers import PerceiverFeatureExtractor, PerceiverForImageClassificationLearned

feature_extractor = PerceiverFeatureExtractor.from_pretrained("deepmind/vision-perceiver-learned")
model = PerceiverForImageClassificationLearned.from_pretrained("deepmind/vision-perceiver-learned")

n_params = sum(p.numel() for p in model.parameters())
print(f'Number of parameters: {n_params:,}')

![](../images/pipeline.png)

## [Example on HuggingFace's space](https://huggingface.co/deepmind/vision-perceiver-learned#how-to-use)

In [ ]:
import requests
from PIL import Image

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

# prepare input
encoding = feature_extractor(image, return_tensors="pt")
inputs = encoding.pixel_values

# forward pass
outputs = model(inputs)
logits = outputs.logits
print("Predicted class:", model.config.id2label[logits.argmax(-1).item()])
image

### Pre-processing and model inference in one function call

In [ ]:
def image2class(image):
    """
    Convenient function to package pre-processsing
    and model inference in one function
    """
    # image pixels into normalize values
    encoding = feature_extractor(image, return_tensors="pt")

    # forward pass
    outputs = model(encoding.pixel_values)
    logits = outputs.logits
    return model.config.id2label[logits.argmax(-1).item()]

### Auxilliary function `inverse_feature_extractor` to reverse the `feature_extractor` step

![](../images/pipeline2.png)

![](images/pipeline2.png)

In [ ]:
import torch
from torch import Tensor
from torchvision import transforms

def inverse_feature_extractor(x: Tensor):
    """"""
    t = x.squeeze(0)

    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]
    t = t.mul(torch.FloatTensor(std).view(3,1,1)).add(torch.FloatTensor(mean).view(3,1,1)) #.numpy()

    im = transforms.ToPILImage()(t)

    return im

inverse_feature_extractor(encoding.pixel_values)

## Let's break the Perceiver with an adversarial example

We use one of the simplest and fastest methods to generate adversarial examples: the fast gradient sign method.

If you wanna get into the details on how it works, read my full article here
http://datamachines.xyz/2021/07/05/adversarial-examples-to-break-deep-learning-models/

In [ ]:
import numpy as np

from cleverhans.torch.attacks.fast_gradient_method import fast_gradient_method

eps = 0.1

# load image
url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)
x = feature_extractor(image, return_tensors="pt").pixel_values

# fast gradient sign method
x_fgm = fast_gradient_method(
    lambda x: model(x).logits, # maps normalized image to model logits
    x, # original image we want to modify
    eps, # max size of the perturbation
    np.inf #l-infinite distance to measure the perturbation
)

# print results
new_image = inverse_feature_extractor(x_fgm)
print('Predicted class: ', image2class(new_image))
new_image